In [1]:
import pandas as pd
import mysql.connector

#### Datasets
1. student
2. student_courses
3. course

In [2]:
df1=pd.read_csv('student.csv')
df1

,Student_id,Name
0,101,ravi
1,102,kiran
2,103,kedar
3,104,manoj
4,105,kruthik
5,106,guru
6,107,darshan
7,108,jithendra
8,109,tarun
9,110,rakshith


In [31]:
df2=pd.read_csv('student_courses.csv')
df2

,Student_id,Course_id
0,101,18ME001
1,104,18ME001
2,106,18ME001
3,110,18ME002
4,102,18ME003
5,105,18ME003
6,108,18ME005
7,109,18ME005
8,103,18ME012
9,107,18ME012


In [36]:
df3=pd.read_csv('course.csv')
df3

,Course_id,Course,Faculty,Points
0,18ME001,Engineering_Maths,Dr.Shrikath,4
1,18ME002,ThermoDynamics,Dr.Basavaraja,3
2,18ME003,CAD,Dr.Mallikarjun,3
3,18ME005,MOM,Sandeep,2
4,18ME012,TurboDynamics,Dr.Rajkumar,2


#### Connecting to the db server

In [5]:
conn=mysql.connector.connect(host='localhost',user='root',password='Hari@8622',autocommit=True)
cursor=conn.cursor()
print("Connected to MySQL Server version %s" % (conn.get_server_info()))


Connected to MySQL Server version 8.0.35


#### Creating Database and Tables with above datasets respectively.

In [6]:
# Execute SQL query to create database
try:
    cursor.execute("CREATE DATABASE GmUniversity")
except  Exception as e:
    print(e)

1007 (HY000): Can't create database 'gmuniversity'; database exists


In [7]:
conn=mysql.connector.connect(host='localhost',user='root',password='Hari@8622',database='GmUniversity',autocommit=True)
cursor=conn.cursor()
print("Connected to MySQL Server version %s" % (conn.get_server_info()))

Connected to MySQL Server version 8.0.35


In [17]:

try:
    cursor.execute("CREATE TABLE student (Student_id int PRIMARY KEY,Name varchar(50))")
    cursor.execute("CREATE TABLE student_courses (Student_id int,Couse_id int)")
    cursor.execute("CREATE TABLE course (Course_id int,Course varchar(100),Faculty varchar(100),Points int)")
except Exception as e:
    print(f"Error creating table : {e}")

Error creating table : 1050 (42S01): Table 'student' already exists


In [29]:
cursor.execute("ALTER TABLE student_courses\
               RENAME COLUMN Couse_id to Course_id")

In [34]:
cursor.execute("ALTER TABLE student_courses\
               MODIFY course_id varchar(50)")

In [37]:
cursor.execute("ALTER TABLE course\
               MODIFY course_id varchar(50)")

In [10]:
# Insert data into 'student' table

for index,row in df1.iterrows():
    cursor.execute("INSERT INTO student VALUES(%s,%s)",(row['Student_id'],row['Name']))

In [35]:
# Insert data into student_courses table

for index,row in df2.iterrows():
    cursor.execute("INSERT INTO student_courses VALUES(%s,%s)",(row['Student_id'],row['Course_id']))

In [43]:
# Insert data into course table

for index, row in df3.iterrows():
    course_id = row['Course_id']
    course = row['Course']
    faculty = row['Faculty']

    # Extract the integer value from the 'Points' column (assuming it's an integer)
    points = row['Points']  # Assuming 'Points' has a single integer value

    # Prepared statement with parameter binding for security and efficiency
    sql = "INSERT INTO course (Course_id, Course, Faculty, Points) VALUES (%s, %s, %s, %s)"
    cursor.execute(sql, (course_id, course, faculty, points))


In [50]:
# Make sure to consume the result of the query
cursor.fetchall()

[(101, 'ravi', '18ME001', 'Engineering_Maths', 'Dr.Shrikath', 4),
 (104, 'manoj', '18ME001', 'Engineering_Maths', 'Dr.Shrikath', 4),
 (106, 'guru', '18ME001', 'Engineering_Maths', 'Dr.Shrikath', 4),
 (110, 'rakshith', '18ME002', 'ThermoDynamics', 'Dr.Basavaraja', 3),
 (102, 'kiran', '18ME003', 'CAD', 'Dr.Mallikarjun ', 3),
 (105, 'kruthik', '18ME003', 'CAD', 'Dr.Mallikarjun ', 3),
 (108, 'jithendra', '18ME005', 'MOM', 'Sandeep', 2),
 (109, 'tarun', '18ME005', 'MOM', 'Sandeep', 2),
 (103, 'kedar', '18ME012', 'TurboDynamics', 'Dr.Rajkumar', 2),
 (107, 'darshan', '18ME012', 'TurboDynamics', 'Dr.Rajkumar', 2)]

In [51]:
tbl1=pd.read_sql_query("SELECT a.Student_id,a.Name,b.Course_id,c.Course,c.Faculty,c.Points\
               FROM student a\
               JOIN student_courses b ON a.Student_id = b.Student_id\
               JOIN course c ON c.Course_id=b.Course_id",con=conn)

C:\Users\Hariprasad Kulkarni\AppData\Local\Temp\ipykernel_5768\968820451.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tbl1=pd.read_sql_query("SELECT a.Student_id,a.Name,b.Course_id,c.Course,c.Faculty,c.Points\


In [52]:
tbl1

,Student_id,Name,Course_id,Course,Faculty,Points
0,101,ravi,18ME001,Engineering_Maths,Dr.Shrikath,4
1,104,manoj,18ME001,Engineering_Maths,Dr.Shrikath,4
2,106,guru,18ME001,Engineering_Maths,Dr.Shrikath,4
3,110,rakshith,18ME002,ThermoDynamics,Dr.Basavaraja,3
4,102,kiran,18ME003,CAD,Dr.Mallikarjun,3
5,105,kruthik,18ME003,CAD,Dr.Mallikarjun,3
6,108,jithendra,18ME005,MOM,Sandeep,2
7,109,tarun,18ME005,MOM,Sandeep,2
8,103,kedar,18ME012,TurboDynamics,Dr.Rajkumar,2
9,107,darshan,18ME012,TurboDynamics,Dr.Rajkumar,2
